In [202]:
import sys

!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install scikit-image
!{sys.executable} -m pip install pillow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [203]:
import ipywidgets as widgets
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
import os

In [204]:
def preprocess_image(image):
    # Resize gambar ke ukuran yang seragam
    image = cv.resize(image, (128, 128))
    
    # Konversi gambar ke grayscale
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    # Normalisasi gambar
    gray_image = cv.normalize(gray_image, None, 0, 255, cv.NORM_MINMAX)
    
    # Penerapan Thresholding Otsu
    _, threshold_image = cv.threshold(gray_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    # Deteksi tepi menggunakan metode Canny
    edges = cv.Canny(threshold_image, 100, 200)
    
    # Ekstraksi fitur HOG dengan parameter yang lebih baik
    features, _ = hog(edges, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    
    return features

In [205]:
def load_dataset_from_folders(dataset_path):
    features = []
    labels = []
    
    # Mendapatkan daftar label berdasarkan nama folder
    label_names = os.listdir(dataset_path)
    
    for label in label_names:
        label_folder = os.path.join(dataset_path, label)
        
        # Pastikan hanya membaca folder (bukan file lain)
        if os.path.isdir(label_folder):
            for img_name in os.listdir(label_folder):
                img_path = os.path.join(label_folder, img_name)
                
                # Baca gambar
                img = cv.imread(img_path)
                
                # Pastikan gambar terbaca dengan benar
                if img is not None:
                    # Preprocessing dan ekstraksi fitur
                    img_features = preprocess_image(img)
                    features.append(img_features)
                    labels.append(label)
                else:
                    print(f"Warning: Gambar {img_path} tidak dapat dibaca.")
    
    return np.array(features), np.array(labels)

In [206]:
# Path ke folder dataset
dataset_path = 'train/'  # Ganti dengan path sebenarnya

# Memuat fitur dan label
X, y = load_dataset_from_folders(dataset_path)

In [207]:
#  Konversi label ke bentuk numerik menggunakan LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [208]:
# Bagi dataset menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Melatih model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [209]:
# Prediksi pada testing set
y_pred = svm_model.predict(X_test)

In [210]:
# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 44.44%


In [211]:
def predict_currency(image_path, model, label_encoder):
    img = cv.imread(image_path)
    
    if img is None:
        print(f"Error: Gambar {image_path} tidak dapat dibaca.")
        return None
    
    features = preprocess_image(img)
    features = features.reshape(1, -1)
    
    prediction = model.predict(features)
    predicted_label = label_encoder.inverse_transform(prediction)

    return predicted_label[0]


In [215]:
# Path ke gambar baru yang ingin diprediksi
new_image_path = 'test/20k.jpg'  # Ganti dengan path sebenarnya

# Prediksi nilai uang
predicted_value = predict_currency(new_image_path, svm_model, le)
print(f"Predicted Currency Value: Rp{predicted_value}")

Predicted Currency Value: Rp20000
